In [1]:
import audio_processing as ap
import audio_utils as au
import math
import numpy.fft as fft
import matplotlib.pyplot as plt
%matplotlib inline

# CS 328 Final Project
## Thomas Bertschinger and Sanders McMillan
### Lab Notebook

### Tuesday, April 26, 2016 **(joint entry)**

Our final project will (likely) be *model* focused rather than experiment focused. We will "implemenet computational model(s) and compare results to existing human data." In this case, the human data will be musical compositions created by humans (*e.g.* Bach fugues) and our model will be a system that "composes" music (perhaps learning from human compositions). 

We set up a bibliography using Latex and BibTex, and a git repository to keep track of our code and materials. 

https://github.com/bertschingert/cs328-final/tree/master/references

### Wednesday, April 27, 2016 (joint entry)

We are thinking about doing a signal-processing based model that creates a representation of a signal and can classify the signal into categories such as instrument family, voice, etc. We will need to create a representation of audio sound in a (hopefully) small number of dimensions that includes enough relevant information so that we can classify a sound into a category such as brass versus string instrument, for example. 

The representation will likely include attributes such as attack and decay time of the waveform; the spectrum at various points in time; how much the spectrum changes over time; irregularities in the spectrum. 

Our model will ideally be able to classify instruments correctly at different amplitudes and pitches. It would also be important to limit the model to audio information that humans can actually percieve. (For example, it wouldn't make sense for our model to take into account frequences substantially above 20,000Hz because humans cannot hear that high.) 

Being able to pull out the relevant information from an audio signal is important because it will help us understand how humans can do things such as distinguish different people's voices. We know probably hundreds of different voices that we can identify from only a few words of speech. This is also important for being able to recognize different instruments present in a single audio signal. 

We think it would be plausible to create a neural network to identify different audio signals. We will first take an audio signal and use tools such as the discrete Fourier transform to create a suitable representation of the signal that omits extraneous information or information that humans cannot perciever. Then, we train a neural network to be able to identify, from the features of the representation, what category the signal belongs to. 

Theoretically, our model will be grounded in the similarity models learned earlier in this course, in addition to Gibsonian and Gestalt principles of perceiving invariants in stimuli and grouping similar and proximal stimuli as belonging to the same perceptual unit (e.g. being able to classify a signal as being of a certain category regardless of it's amplitude and signal, and classifying/perceiving similar successive waveforms as being from the same instrument). It will also involve the place and time theories for how humans transform air pressure hitting the ear into an auditory representation (which is where our Fourier transform and representation stages come in). 

### Monday, May 16, 2016 (joint entry)

We have written some code to do basic audio processing (FFT). We are also starting to create the NN..

In order to keep the neural network consistent, we will have the option to save weights and biases to a text file so that they can be loaded. 

In [2]:
left, right = au.read_raw_stereo("audio_files/violin-a440.raw")
chunk = left[:44100]
freqs = fft.rfft(chunk)
au.graph_fft(freqs[:10000])

FileNotFoundError: [Errno 2] No such file or directory: 'audio_files/violin-a440.raw'

We also started compiling our audio sample library. The audio samples are 1.5 second long clips downloaded from http://www.philharmonia.co.uk/explore/make_music. So far we have 5 different instruments (guitar, saxophone, flute, violin, and trumpet), each at three different pitches (A, C, and E). The octaves of the pitches are different for each instrument as the library did not contain all octaves for each instrument, and each instrument has different pitch restraints.

### Tuesday, May 17, 2016 (joint entry)

We are starting to write code that uses the FFT to get some information on the signal, such as attack time. This will be helpful for our representation. We are also starting to implement the neural network. 

### Saturday, May 21, 2016

Created the Hann window function. 

In [ ]:
test_signal = []
for i in range(100):
    test_signal.append(1)
w = ap.hann_window(test_signal)
au.graph_signal(w)

Created code to compute the spectral centroid

In [ ]:
wave = au.create_sine_wave(440, 1, 1)
spectrum = fft.rfft(wave)
print("before window: spectral centroid is ", ap.get_spectral_centroid(spectrum, 44100))
au.graph_fft(spectrum[:500])

In [ ]:
w_signal = ap.hann_window(wave)
w_spectrum = fft.rfft(w_signal)
print("after window: spectral centroid is ", ap.get_spectral_centroid(w_spectrum, 44100))
au.graph_fft(w_spectrum[:500])

### Sunday, May 22, 2016

Now using the python wave library to read .wav files. Created the function read_wav_mono in audio_utils.py which returns a list of the samples. 

In [ ]:
f = au.read_wav_mono('audio_files/guitar_A4_very-long_forte_normal.wav')
au.graph_signal(f)

In [ ]:
step= int(44100 / 4)
start = 44100
chunk = f[start:start+step]
s = fft.rfft(chunk)
au.graph_fft(s, 1000)

In [ ]:
w = ap.hann_window(chunk)
s = fft.rfft(w)
au.graph_fft(s, 1000)

In [ ]:
s = ap.spectral_flux(f)
au.graph_signal(s)

In [ ]:
f2 = au.read_wav_mono('audio_files/saxophone_A4_15_forte_normal.wav')
au.graph_signal(f2)

In [ ]:
s2 = ap.spectral_flux(f2)
au.graph_signal(s2)
for i in s2:
    print(i)

### Thursday, May 26

We finished writing code for our neural network, and are now testing and de-bugging it. Our neural network has an initialize_network function that initializes the weights and nodes of the networks based on input length, the output length, the number of hidden units, and the number of layers of the network. It also has a set_hidden_units function that allows you to set the number of hidden units at any particular hidden layer, and changes the weights accordingly.

In [1]:
# Testing the initialize network function
import neural_net as nn
import numpy as np
nn.initialize_network(4, 3, 2, 3)

[array([[ 0.53583157,  0.37605029,  0.64027522,  0.42711325],
        [ 0.99766032,  0.78736334,  0.50272463,  0.68582115],
        [ 0.83431295,  0.3429505 ,  0.07815369,  0.36288005]]),
 array([[ 0.82471077,  0.2414166 ,  0.87454694],
        [ 0.02866854,  0.70695496,  0.4506752 ],
        [ 0.11439646,  0.72454449,  0.53050173]])]

In [2]:
#Testing the set hidden units function
nn.set_hidden_units(1, 5)

[array([[ 0.46905087,  0.57140433,  0.44536742,  0.43432284],
        [ 0.59206108,  0.30714227,  0.37903945,  0.96927626],
        [ 0.7412559 ,  0.75477984,  0.9881189 ,  0.44569425],
        [ 0.26689418,  0.16495126,  0.04135824,  0.62399558],
        [ 0.47180655,  0.84089444,  0.41694873,  0.21328739]]),
 array([[ 0.82075489,  0.59253942,  0.45404062,  0.11430654,  0.85821336],
        [ 0.40295664,  0.65716793,  0.82903873,  0.55309532,  0.18747298],
        [ 0.77588933,  0.71952014,  0.26807361,  0.81921477,  0.6340558 ]])]

In [3]:
#Testing the update weights function
input = np.array([[1,2,3, 4], [5, 6, 7, 8], [9,10,11, 12]])
output = np.array([[1, 0, 0], [0,1,0], [0,0,1]])
np.array(output)
nn.update_weights(input.T, output.T, 0.01)
print(nn.get_weights())

Input:  [[1]
 [2]
 [3]
 [4]]
activation at layer:  1 [[ 0.99483795]
 [ 0.9982188 ]
 [ 0.99962178]
 [ 0.96174899]
 [ 0.99369691]]
activation at layer:  2 [[ 0.9713287 ]
 [ 0.93548685]
 [ 0.96809513]]
New weights at layer  1 [[ 0.82076283  0.59254739  0.45404861  0.11431422  0.85822129]
 [ 0.40239498  0.65660436  0.82847436  0.55255234  0.18691196]
 [ 0.77559186  0.71922166  0.2677747   0.81892719  0.63375867]]
New weights at layer  0 [[ 0.46904855  0.57139968  0.44536045  0.43431354]
 [ 0.59206005  0.3071402   0.37903635  0.96927213]
 [ 0.7412557   0.75477943  0.98811829  0.44569343]
 [ 0.26687373  0.16491035  0.04129689  0.62391377]
 [ 0.47180474  0.84089083  0.41694332  0.21328017]]
Input:  [[5]
 [6]
 [7]
 [8]]
activation at layer:  1 [[ 0.9999976 ]
 [ 0.99999978]
 [ 1.        ]
 [ 0.99950573]
 [ 0.99999733]]
activation at layer:  2 [[ 0.97174988]
 [ 0.93681787]
 [ 0.96926408]]
New weights at layer  1 [[ 0.82049607  0.59228062  0.45378184  0.11404758  0.85795453]
 [ 0.40243238  0.6566

In [4]:
(weights, bias) = nn.train_network(input.T, output.T, nn.update_weights, 25, 0.1)
print(weights)

Iteration:  0
Input:  [[1]
 [2]
 [3]
 [4]]
activation at layer:  1 [[ 0.99483758]
 [ 0.99821874]
 [ 0.99962178]
 [ 0.96172533]
 [ 0.99369656]]
activation at layer:  2 [[ 0.97124135]
 [ 0.93509846]
 [ 0.96798851]]
New weights at layer  1 [[ 0.82030884  0.59209367  0.453595    0.1138577   0.85776721]
 [ 0.39623234  0.65042253  0.82228458  0.5465776   0.1807558 ]
 [ 0.77232829  0.71594795  0.26449678  0.81576308  0.63049852]]
New weights at layer  0 [[ 0.46902544  0.57135346  0.44529112  0.43422111]
 [ 0.59204975  0.30711961  0.37900545  0.96923094]
 [ 0.74125365  0.75477533  0.98811213  0.44568523]
 [ 0.26666919  0.16450178  0.04068428  0.62309714]
 [ 0.4717869   0.84085514  0.41688979  0.2132088 ]]
Input:  [[5]
 [6]
 [7]
 [8]]
activation at layer:  1 [[ 0.9999976 ]
 [ 0.99999978]
 [ 1.        ]
 [ 0.99949851]
 [ 0.99999732]]
activation at layer:  2 [[ 0.97167494]
 [ 0.93460079]
 [ 0.96867711]]
New weights at layer  1 [[ 0.81763453  0.58941935  0.45092068  0.11118472  0.8550929 ]
 [ 0.39

In [7]:
#Testing predict network function
inp = np.array([[1,2,3, 4]])
print(nn.predict_network(inp.T, nn.logistic))

activation at layer:  1 [[ 0.99473557]
 [ 0.99820032]
 [ 0.999621  ]
 [ 0.95260493]
 [ 0.99364154]]
activation at layer:  2 [[ 0.90438564]
 [ 0.50900569]
 [ 0.87503054]]
[[ 0.90438564]
 [ 0.50900569]
 [ 0.87503054]]


In [7]:
zcr_data = [np.loadtxt("reps/clarinet_zcr.txt"), np.loadtxt("reps/flute_zcr.txt"), np.loadtxt("reps/guitar_zcr.txt"), np.loadtxt("reps/saxophone_zcr.txt"), np.loadtxt("reps/violin_zcr.txt")]
sc_data = [np.loadtxt("reps/clarinet_centroid.txt"), np.loadtxt("reps/flute_centroid.txt"), np.loadtxt("reps/guitar_centroid.txt"), np.loadtxt("reps/saxophone_centroid.txt"), np.loadtxt("reps/violin_centroid.txt")]
# f_data = 
# g_data = 
# s_data = 
# v_data = 
inp = []
inputs = []
outputs = []
for i in range(100):
    for j in range(5):
        inputs = []
        output = [0]*5
        inputs.append(zcr_data[j][i])
        inputs.append(sc_data[j][i])
        output[j] = 1
        outputs.append(output)
        inp.append(np.array(inputs))
        # for k in range(5):
        #    if k != j:
         #       outputs[k].append(0)
# print(inp)
inp = np.array(inp)
print(inp.T)
output = np.array(outputs)
print(output)
test_inp = np.array([inp[0]])
nn.initialize_network(2, 5, 3, 5)
nn.train_network(inp.T, output.T, nn.update_weights, 1000, 0.1)
print(nn.predict_network(test_inp.T, nn.logistic))

[[  4.15419501e-02   2.72108844e-02   5.07936508e-03   2.11337868e-02
    2.10430839e-02   5.49659864e-02   1.86848073e-02   1.17913832e-02
    2.79365079e-02   2.43990930e-02   2.39455782e-02   1.32426304e-02
    1.24263039e-02   6.14058957e-02   2.14058957e-02   2.64852608e-02
    1.61451247e-02   2.53968254e-03   2.37641723e-02   2.29478458e-02
    3.83673469e-02   4.11791383e-02   1.41496599e-02   2.32199546e-02
    1.83219955e-02   5.99546485e-02   2.08616780e-02   6.98412698e-03
    2.44897959e-02   1.90476190e-02   2.57596372e-02   2.32199546e-02
    9.88662132e-03   2.88435374e-02   2.14965986e-02   2.43990930e-02
    1.80498866e-02   4.89795918e-03   6.65759637e-02   2.59410431e-02
    3.93650794e-02   1.50566893e-02   2.04081633e-02   2.76643991e-02
    2.19501134e-02   4.25396825e-02   3.16553288e-02   6.80272109e-03
    2.73922902e-02   1.92290249e-02   2.17687075e-02   2.28571429e-02
    1.98639456e-02   5.50566893e-02   2.49433107e-02   3.02947846e-02
    2.35827664e-02  

In [ ]:
#Testing the neural network on the data from ps6



### Thursday, May 26

Our neural network code is not predicting correctly, and I am spending my time going back to try and de-bug it.